## Prophet Water Level Outliers

Use water level measurements of groundwater available from IntellusNM.com to identify outliers, trend, seasonality, and upset events.

This notebook contains basic statistical analysis and visualization of the data.

### Data Sources
- summary : Processed file from notebook 1-Data_Prep

### Changes
- 02-19-2024 : Started project

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import seaborn as sns
import prophet
import plotly
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


In [ ]:
%matplotlib inline

### File Locations

In [ ]:
today = datetime.today()
in_file = Path.cwd() / "data" / "processed" / f"summary_{today:%b-%d-%Y}.pkl"
report_dir = Path.cwd() / "reports"
report_file = report_dir / "Excel_Analysis_{today:%b-%d-%Y}.xlsx"

In [ ]:
df = pd.read_pickle(in_file)
df = df.rename(columns={'Measurement Date Time':'ds', 'Groundwater Elevation':'y'})
df.dtypes

In [ ]:
pip show prophet

### Perform Data Analysis - Loop through locations to identify anomalies

In [ ]:
location = df[['Location ID','Site ID']].drop_duplicates().reset_index()

In [ ]:
markers = {'N':'X', 'Y':'o'}
hue_order = ['N','Y']
style_order = ['N','Y']
iw = 0.99


for location, site_id in zip(location['Location ID'], location['Site ID']):
	# Add seasonality and instantiate a new Prophet model
	model = Prophet(interval_width=iw, yearly_seasonality=True, weekly_seasonality=True)

	# print(location, parameter)
	export_subset = df[(df['Location ID'] == location) & (df['Site ID'] == site_id)]
	
	export_subset = export_subset[export_subset.groupby(['Location ID']).transform('size')>10]

	if export_subset.empty:
		continue

	# Fit the model on the training dataset
	model.fit(export_subset)

	# Make prediction
	forecast = model.predict(export_subset)

	# Merge actual and predicted values
	performance = pd.merge(export_subset, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')

	# Create an anomaly indicator
	performance['anomaly'] = performance.apply(lambda rows: 1 if ((rows.y<rows.yhat_lower)|(rows.y>rows.yhat_upper)) else 0, axis = 1)

	anomalies = performance[performance['anomaly']==1].sort_values(by='ds')
	if anomalies.empty:
		continue
	
	anomalies.to_csv('anomalies.csv', mode='a', index=False, header=False)

### Upload anomalies and prepare for graphing of results with anomalies identified

In [ ]:
outlier_location_parameter = pd.read_csv('anomalies.csv', header=None, names = ('site_id','location_id','sample_date_time','groundwater_measurement','groundwater_elevation','groundwater_level_comment','groundwater_level_data_quality_code','groundwater_level_data_quality_reason_code','yhat','yhat_lower','yhat_upper','anomaly'))
outlier_location_parameter['sample_date_time'] = pd.to_datetime(outlier_location_parameter['sample_date_time'])
outlier_location_parameter = outlier_location_parameter[outlier_location_parameter['sample_date_time']>pd.to_datetime('2018-05-01')]
outlier_location_parameter = outlier_location_parameter[['location_id','site_id','sample_date_time']].drop_duplicates()
# outlier_location_parameter.to_csv('recent_anomalies.csv', index=False, header=False)
outlier_location_parameter.dtypes

In [ ]:
markers = {'N':'X', 'Y':'o'}
hue_order = ['N','Y']
style_order = ['N','Y']


for location, site in zip(outlier_location_parameter['location_id'],outlier_location_parameter['site_id']):
	# Add seasonality and instantiate a new Prophet model
	model = Prophet()
	model = Prophet(interval_width=iw, yearly_seasonality=True, weekly_seasonality=True)

	# print(location, parameter)

	export_subset = df[(df['Location ID'] == location) & (df['Site ID'] == site)]

	if export_subset.empty:
		continue

	# Fit the model on the training dataset
	model.fit(export_subset)

	# Make prediction
	forecast = model.predict(export_subset)

	# Merge actual and predicted values
	performance = pd.merge(export_subset, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')

	# Create an anomaly indicator
	performance['anomaly'] = performance.apply(lambda rows: 1 if ((rows.y<rows.yhat_lower)|(rows.y>rows.yhat_upper)) else 0, axis = 1)

	anomalies = performance[performance['anomaly']==1].sort_values(by='ds')
	# if anomalies.empty:
	# 	continue
	
	# Visualize the anomalies
	fig = model.plot(forecast); 
	
	# a = add_changepoints_to_plot(fig.gca(), model, forecast)# Add semi-colon to remove the duplicated chart

	# py.iplot(fig)

	name = (location +' '+ site + ' outliers.png').replace("/","-")
	y_axis_label = site  + " " + 'Groundwater Elevation (ft)'
	plt.ylabel(y_axis_label)
	plt.xlabel('Sample Collection Date')
	plt.suptitle('\n'+location)
	fig.savefig('reports/' + name)